In [9]:
import os
import pandas as pd
import numpy as np
import json
import pickle
from collections import defaultdict
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

import torch

import os, sys
parentPath = os.path.abspath("..")
if parentPath not in sys.path:
    sys.path.insert(0, parentPath)
    
from src.data import load_source, slice_d
from src.config import Config, get_option_fallback
from knowledge_tracing.trainer import Trainer

In [2]:
projectdir = Path('/code')
assert projectdir.exists()

In [22]:
results = [
    'output/1220_gt_all_assistment2009/dkt/report/20191220-2209/ksdkt.json',
    'output/1220_gt_all_assistment2009/dkt_w1w2/report/20191220-2224/ksdkt.json',
    'output/1220_gt_all_assistment2009/dkt_ks05/report/20191220-2238/ksdkt.json',
    'output/1220_gt_all_assistment2009/dkt_pre10/report/20191220-2252/ksdkt.json',
]

In [23]:
df = pd.DataFrame(columns=['dataset', 'model', 'lw1', 'lw2', 'ksv', 'pre', 'auc', 'w1', 'w2', 'ksv_score'])

for res_path in results:
    with open(projectdir / res_path, 'r') as f:
        res = json.load(f)
    best_idx = np.argmax(res['indicator']['eval_auc'])
    row = {
        'dataset':   res['config']['source_data'],
        'model':     res['config']['model_name'],
        'lw1':       res['config']['lambda_l1'],
        'lw2':       res['config']['lambda_l2'],
        'ksv':       res['config']['ksvector_l1'],
        'pre':       res['config']['pre_dummy_epoch_size'],
        'auc':       res['indicator']['eval_auc'][best_idx],
        'w1':        res['indicator']['waviness_l1'][best_idx],
        'w2':        res['indicator']['waviness_l2'][best_idx],
        'ksv_score': res['indicator']['ksvector_l1'][best_idx],
    }
    df = df.append(row, ignore_index=True)
    
df

,dataset,model,lw1,lw2,ksv,pre,auc,w1,w2,ksv_score
0,original_ASSISTmentsSkillBuilder0910,ksdkt,0.000,0.0,0.0,0,0.848056,0.031516,0.003574,0.010638
1,original_ASSISTmentsSkillBuilder0910,ksdkt,0.003,3.0,0.0,0,0.845646,0.016622,0.000966,0.010267
2,original_ASSISTmentsSkillBuilder0910,ksdkt,0.000,0.0,0.5,0,0.848250,0.030459,0.003318,0.009816
3,original_ASSISTmentsSkillBuilder0910,ksdkt,0.000,0.0,0.0,10,0.849444,0.034220,0.003921,0.010797


In [24]:
print(df.to_latex())

\begin{tabular}{lllrrrlrrrr}
\toprule
{} &                               dataset &  model &    lw1 &  lw2 &  ksv & pre &       auc &        w1 &        w2 &  ksv\_score \\
\midrule
0 &  original\_ASSISTmentsSkillBuilder0910 &  ksdkt &  0.000 &  0.0 &  0.0 &   0 &  0.848056 &  0.031516 &  0.003574 &   0.010638 \\
1 &  original\_ASSISTmentsSkillBuilder0910 &  ksdkt &  0.003 &  3.0 &  0.0 &   0 &  0.845646 &  0.016622 &  0.000966 &   0.010267 \\
2 &  original\_ASSISTmentsSkillBuilder0910 &  ksdkt &  0.000 &  0.0 &  0.5 &   0 &  0.848250 &  0.030459 &  0.003318 &   0.009816 \\
3 &  original\_ASSISTmentsSkillBuilder0910 &  ksdkt &  0.000 &  0.0 &  0.0 &  10 &  0.849444 &  0.034220 &  0.003921 &   0.010797 \\
\bottomrule
\end{tabular}



In [35]:
resultpath2 = sorted(projectdir.glob('output/1221_gt_searchPreX_assistment2009/*/report/*/*.json'))

In [37]:
df = pd.DataFrame(columns=['dataset', 'model', 'lw1', 'lw2', 'ksv', 'pre', 'bestepoch', 'auc', 'w1', 'w2', 'ksv_score'])

for res_path in resultpath2:
    with open(projectdir / res_path, 'r') as f:
        res = json.load(f)
    best_idx = np.argmax(res['indicator']['eval_auc'])
    row = {
        'dataset':   res['config']['source_data'],
        'model':     res['config']['model_name'],
        'lw1':       res['config']['lambda_l1'],
        'lw2':       res['config']['lambda_l2'],
        'ksv':       res['config']['ksvector_l1'],
        'pre':       res['config']['pre_dummy_epoch_size'],
        'bestepoch': best_idx * 10,
        'auc':       res['indicator']['eval_auc'][best_idx],
        'w1':        res['indicator']['waviness_l1'][best_idx],
        'w2':        res['indicator']['waviness_l2'][best_idx],
        'ksv_score': res['indicator']['ksvector_l1'][best_idx],
    }
    df = df.append(row, ignore_index=True)
    
df

,dataset,model,lw1,lw2,ksv,pre,bestepoch,auc,w1,w2,ksv_score
0,original_ASSISTmentsSkillBuilder0910,ksdkt,0.0,0.0,0.0,0,790,0.848056,0.031516,0.003574,0.010638
1,original_ASSISTmentsSkillBuilder0910,ksdkt,0.0,0.0,0.0,10,1190,0.850610,0.036189,0.004302,0.010970
2,original_ASSISTmentsSkillBuilder0910,ksdkt,0.0,0.0,0.0,100,1130,0.852440,0.044055,0.006467,0.011094
3,original_ASSISTmentsSkillBuilder0910,ksdkt,0.0,0.0,0.0,150,990,0.851169,0.045570,0.007067,0.011021
4,original_ASSISTmentsSkillBuilder0910,ksdkt,0.0,0.0,0.0,200,1080,0.854232,0.045542,0.007027,0.011062
5,original_ASSISTmentsSkillBuilder0910,ksdkt,0.0,0.0,0.0,15,1090,0.850954,0.035666,0.004176,0.010684
6,original_ASSISTmentsSkillBuilder0910,ksdkt,0.0,0.0,0.0,20,600,0.849863,0.033929,0.003678,0.010769
7,original_ASSISTmentsSkillBuilder0910,ksdkt,0.0,0.0,0.0,30,890,0.851544,0.039166,0.004868,0.010808
8,original_ASSISTmentsSkillBuilder0910,ksdkt,0.0,0.0,0.0,40,1150,0.853078,0.042221,0.005736,0.010904
9,original_ASSISTmentsSkillBuilder0910,ksdkt,0.0,0.0,0.0,5,700,0.849700,0.030599,0.003359,0.010709


In [38]:
print(df.to_latex())

\begin{tabular}{lllrrrllrrrr}
\toprule
{} &                               dataset &  model &  lw1 &  lw2 &  ksv &  pre & bestepoch &       auc &        w1 &        w2 &  ksv\_score \\
\midrule
0  &  original\_ASSISTmentsSkillBuilder0910 &  ksdkt &  0.0 &  0.0 &  0.0 &    0 &       790 &  0.848056 &  0.031516 &  0.003574 &   0.010638 \\
1  &  original\_ASSISTmentsSkillBuilder0910 &  ksdkt &  0.0 &  0.0 &  0.0 &   10 &      1190 &  0.850610 &  0.036189 &  0.004302 &   0.010970 \\
2  &  original\_ASSISTmentsSkillBuilder0910 &  ksdkt &  0.0 &  0.0 &  0.0 &  100 &      1130 &  0.852440 &  0.044055 &  0.006467 &   0.011094 \\
3  &  original\_ASSISTmentsSkillBuilder0910 &  ksdkt &  0.0 &  0.0 &  0.0 &  150 &       990 &  0.851169 &  0.045570 &  0.007067 &   0.011021 \\
4  &  original\_ASSISTmentsSkillBuilder0910 &  ksdkt &  0.0 &  0.0 &  0.0 &  200 &      1080 &  0.854232 &  0.045542 &  0.007027 &   0.011062 \\
5  &  original\_ASSISTmentsSkillBuilder0910 &  ksdkt &  0.0 &  0.0 &  0.0 &   15 &